In [18]:
import re
from selenium import webdriver
from selenium.webdriver.common.keys import Keys

In [2]:
import pandas as pd

# Crawl stock price

## Selenium

In [19]:
# Historical price of IBM quote
url = 'https://www.investing.com/equities/ibm-historical-data'

In [20]:
browser = webdriver.Firefox(executable_path='geckodriver.exe')

In [21]:
browser.get(url)

### Close popup

In [7]:
'''
Popup depend on guest activity
It don't affect the crawling
'''
# browser.find_element_by_class_name('popupCloseIcon.largeBannerCloser').click()

### Click the calendar

In [22]:
browser.find_element_by_id('widgetFieldDateRange').click()

### Find and input start date to calendar

In [23]:
startDate = browser.find_element_by_id('startDate')
startDate.clear()
startDate.send_keys('02/15/2001')
browser.find_element_by_id('applyBtn').send_keys(Keys.ENTER)

### Look up value

In [39]:
'''
Find the real volume number
Eg: <td data-real-value="114876432">114.88M</td>
With the volume, we need to take 114876432 from the source code
not the text between <td> tag
'''

page_source = browser.page_source
volume = re.findall(r'(data-real-value=")(\d+)(">\d+.\d+[BM])', page_source)
len(volume)

5000

In [40]:
str_xpath = '/html/body/div[5]/section/div[9]/table[1]/tbody/tr[{}]/td[{}]'

In [41]:
date = []
close = []
Open = []
high = []
low = []
vol = []
change =[]
index = [1, 2, 3, 4, 5, 7]

In [42]:
%%time
'''
Find other fields of the table with text between <td> tag
'''
for i in range(1, 5000+1):
    vol.append(volume[i - 1][1])
    for j in index:
        xpath = str_xpath.format(i, j)
        real_value = browser.find_element_by_xpath(xpath)
        if j == 1:
            date.append(real_value.text)
        if j == 2:
            close.append(real_value.text)
        if j == 3:
            Open.append(real_value.text)
        if j == 4:
            high.append(real_value.text)
        if j == 5:
            low.append(real_value.text)
        if j == 7:
            change.append(real_value.text.strip('%'))

Wall time: 7min 46s


In [67]:
browser.quit()

In [43]:
len(date)

5000

In [44]:
len(close)

5000

In [45]:
len(Open)

5000

In [46]:
len(high)

5000

In [47]:
len(low)

5000

## Save dataframe

In [49]:
stock_price = pd.DataFrame({
    'Date': date, 
    'Close': close, 
    'Open': Open, 
    'High': high, 
    'Low': low,
    'Vol': vol,
    'Change_%': change
    })
stock_price

,Date,Close,Open,High,Low,Vol,Change_%
0,"Dec 30, 2020",124.34,123.74,124.82,123.63,3380494,0.44
1,"Dec 29, 2020",123.80,125.25,125.48,123.25,3487007,-0.82
2,"Dec 28, 2020",124.82,125.12,126.57,124.46,3615222,0.10
3,"Dec 24, 2020",124.69,125.00,125.10,124.21,1761122,0.64
4,"Dec 23, 2020",123.90,123.93,125.16,123.89,2693889,0.23
...,...,...,...,...,...,...,...
4995,"Feb 22, 2001",108.90,106.50,110.90,103.38,11709500,1.29
4996,"Feb 21, 2001",107.51,109.55,111.50,107.29,9158200,-3.58
4997,"Feb 20, 2001",111.50,114.55,115.60,110.78,6748300,-3.04
4998,"Feb 16, 2001",115.00,114.55,115.75,113.94,7073000,-1.52


In [50]:
stock_price.to_csv('stock_price.csv', index=False)

In [133]:
df = pd.read_csv('stock_price.csv', index_col=False)

In [52]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5000 entries, 0 to 4999
Data columns (total 7 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   Date      5000 non-null   object 
 1   Close     5000 non-null   float64
 2   Open      5000 non-null   float64
 3   High      5000 non-null   float64
 4   Low       5000 non-null   float64
 5   Vol       5000 non-null   int64  
 6   Change_%  5000 non-null   float64
dtypes: float64(5), int64(1), object(1)
memory usage: 273.6+ KB
